# Part 2, Topic 1: Fault Attack on RSA (MAIN)

## Acknowledgements

This attack is was originally detailed in a [1997 paper by Boneh, Demillo, and Lipton](https://www.researchgate.net/publication/2409434_On_the_Importance_of_Checking_Computations). This tutorial draws heavily from a blog post by David Wong, which you can find [here](https://www.cryptologie.net/article/371/fault-attacks-on-rsas-signatures/).

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *So far, we've seen how clock and voltage glitches can be used to disrupt a microcontroller or FPGA, potentially changing the execution path. We've also seen that, in the case of AES, if we disrupt the calculation in a certian spot only twice, we can easily recover a quarter of the key bytes.*

*In this lab, we'll explore an attack on RSA, more specifically an RSA optimization, that can recover the entire key with a single fault.*

**LEARNING OUTCOMES:**

* Understanding conditions for the fault
* Recovering an RSA private key from a faulty signature

## Attack Theory

The theory for this attack is a bit simpler than the one for AES, so we'll take you through it here instead of putting it in another notebook. If you don't know, RSA is an asymmetric cryptographic algorithm, meaning it has a public and private key, with one being used for encryption and the other for decryption. It can be used in both a public encryption/private decryption configuration and a private encryption/public decryption situation. We'll be attacking the latter case. Let's assume that the target is signing a message and is sending it to us so we can verify the identity of the device.

For RSA, the public key is made up of two numbers, $n$ and $e$. The private key is made up of $n$ and $d$. These numbers have some special properties:

1. $n$ is constructed as the product of 2 prime numbers, $p$ and $q$.
1. $d$ is derived from $p$, $q$, and $e$.

As a result of these properties, a few things are evident:

1. If we learn $p$ or $q$, we can derive the other prime number since $n$ is public information.
1. If we learn $p$ and $q$, we can derive $d$ since $e$ is public information.

The target can encrypt/sign a message with the following equation ($s$ is the signature and $m$ is the message):

$$s = m^d({mod}\space n)$$

And we can decrypt/verify the message with the following equation:

$$s^e = m(mod\space n)$$

One issue with RSA is that it's very slow - these equations are simple, but they use massive numbers; if the numbers used are too small, it's trivial for a computer to factor $n$ into $p$ and $q$. $n$ for the firmware we're attacking is 1024 bits long, which is on the smaller size of secure keys at this point. This encryption/signing operation is especially slow.

An important property of the signature verification equation is that the following is also true:

$$(m - s^e)(mod\space n) = 0$$

aka $(m - s^e)$ is divisible by $n$. It follows that $(m - s^e)$ is divisible by $p$ and $q$ as well.

## The Chinese Remainder Theorem (CRT)

To help speed up the encryption, an optimization known as the Chinese Remainder Theorem (CRT) can be used. This theorem allows us derive two new exponents, $d_p$ and $d_q$, which are much smaller than $d$. We can then replace

$$s = m^d({mod}\space n)$$

with two equations

$$s_1 = m^{d_p}(mod\ p) \\
s_2 = m^{d_q}(mod\ q)$$

$s$ is then:

$$i_q = q^{-1}mod\ p \\
s = s_2 + q(i_q(s_1 - s_2)mod\ p)
$$

$s_1$ and $s_2$ can be combined into $s$ via CRT. Even though there's two modular exponentiations, this is still much faster since $d_p$ and $d_q$ are much smaller than $d$ and $p$ and $q$ are much smaller than $n$. This is the optimization that our target, which is using a slightly modified version of MBEDTLS (more on that in a bit), makes.

## Inserting a Fault

Suppose that instead of everything going smoothly as above, a fault happens during the calculation of $s_2$, turning it into $s'_2$. $s_1$ and $s'_2$ will then be combined into $s'$. If that happens, a couple of things will be true:

1. When we go to verify the signature, it won't work: $s'^e \neq m (mod\ n)$. Concequently, $(m - s'^e)(mod\space n) \neq 0$, so $(m - s'^e)$ is not divisible by $n$
1. Since $(m - s'^e)$ is not divisible by $n$, it cannot be divisible by both $p$ and $q$ anymore. Due to how the CRT works, it will still be divisible by $p$, but not by $q$. This can be expressed as $(m - s'^e) = pk$ for some integer $k$.  
1. We now have $(m - s'^e) = pk$ and $n = pq$. This means that we can find $p$ by computing $gcd(m-s'^e, n)$
    
Once we have $p$, we can compute $q = \frac{n}{p}$ and $d$ via the rest of the RSA key generation algorithm. 

The math is similar in the case that $s_1$ is faulted, with the only difference being that we recover $q$ at the end instead of $p$.

## Communicating with the Target

With the math out of the way, we can get to attacking the target:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
SS_VER='SS_VER_1_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-rsa
make PLATFORM=$1 CRYPTO_TARGET=MBEDTLS CRYPTO_OPTIONS=RSA OPT=2 SS_VER=$2

In [ ]:
cw.target_logger.setLevel(cw.logging.WARNING)

In [ ]:
import time
fw_path = "../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
time.sleep(1)

This target is using the simpleserial protocol, but the full signature is too big to read back in a single command. This means we instead read back the signature in 3 commands:

1. `'t'` will do the signature calculation and respond with the first 48 bytes of the signature
1. `'1'` will return the next 48 bytes of the signature
1. `'2'` will return the final 32 bytes of the signature

In [ ]:
import time
scope.clock.adc_src = "clkgen_x1"
reset_target(scope)
target.flush()
scope.arm()
target.simpleserial_write("t", bytearray([]))
    
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')
    
time.sleep(2)
if SS_VER=='SS_VER_2_1':
    output = target.simpleserial_read_witherrors('r', 128, timeout=1)
else:
    output = target.simpleserial_read_witherrors('r', 48, timeout=1)

In [ ]:
sig = None
if output['valid']:
    print(output['payload'])
    sig = output['payload']
    
print(scope.adc.trig_count)

That took a long time, probably more than 10M cycles! Let's read back the rest of the message and append it to our signature:

In [ ]:
if SS_VER!='SS_VER_2_1':
    target.simpleserial_write("1", bytearray())
    time.sleep(0.2)
    output = target.simpleserial_read_witherrors('r', 48, timeout=10)
    if output['valid']:
        sig.extend(output['payload'])

    target.simpleserial_write("2", bytearray())
    time.sleep(0.2)
    output = target.simpleserial_read_witherrors('r', 32, timeout=10)
    if output['valid']:
        sig.extend(output['payload'])

    print(sig)

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

Let's verify that our signature is correct and that we can verify it:

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 

from Crypto.Hash import SHA256

e = 0x10001
n = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
m = b"Hello World!"

hash_object = SHA256.new(data=m)
pub_key = RSA.construct((n, e))
signer = PKCS1_v1_5.new(pub_key) 
sig_check = signer.verify(hash_object, sig)
print(sig_check)

assert sig_check, "Failed to verify signature on device. Got: {}".format(newout)

Now let's setup the glitch module. Use settings here that worked for you before. Ideally, you'll have a single group of settings here - RSA is very slow (as you've seen), so trying to scan settings here would take forever!

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
# These width/offset numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1:
scope.glitch.width = -9
scope.glitch.offset = -38.3
scope.io.hs2 = "glitch"
print(scope.glitch)

In [ ]:
from tqdm.notebook import trange
import time
for i in trange(7000000, 7100000): #look for something kind of near the end
    scope.glitch.ext_offset = i
    scope.adc.timeout = 3
    target.flush()
    scope.arm()
    target.simpleserial_write("t", bytearray([]))

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    if SS_VER=='SS_VER_2_0':
        output = target.simpleserial_read_witherrors('r', 128, timeout=100, glitch_timeout=1)
    else: 
        output = target.simpleserial_read_witherrors('r', 48, timeout=100, glitch_timeout=1)
    if invalid_output: # replace with invalid output detection
        print("crash") #we can't really do anything here - we need the full signature back
    else:
        if glitched_output: #detect if the calculation was messed up
            # call the faulty signature whatever you want
            # but we'll assume it's called sig for the rest of the lab
            sig = 
        else:
            pass # normal operation, nothing special

## Padding the Message

We've got our glitched signature, but we've still got a little work to do. The $m$ isn't actually the message by itself. Instead, it's a PKCS#1 v1.5 padded hash of it. Luckily, this standard's fairly simple. PKCS#1 v1.5 padding looks like:

\|00\|01\|ff...\|00\|hash_prefix\|message_hash\|

Here, the ff... part is a string of ff bytes long enough to make the size of the padded message the same as N, while hash_prefix is an identifier number for the hash algorithm used on message_hash. Our message was hashed using SHA256, which has the hash prefix `3031300d060960864801650304020105000420`.

We can get our hashed message and $m$ with the following code:

In [ ]:
from Crypto.Hash import SHA256
from binascii import hexlify, unhexlify

def build_message(m, n):
    sha_id = "3031300d060960864801650304020105000420"
    N_len = (len(bin(n)) - 2 + 7) // 8
    pad_len = (len(hex(n)) - 2) // 2 - 3 - len(m)//2 - len(sha_id)//2
    padded_m = "0001" + "ff" * pad_len + "00" + sha_id + m
    return padded_m

hashed_m = hexlify(hash_object.digest()).decode()
padded_m = build_message(hashed_m, n)
print(hashed_m)
print(padded_m)

## Recovering the Private Key

Now we can recover the private values by plugging them into the equations from earlier. If you can, install the gmpy2 Python library, which has much better support for big integer math like this. Otherwise, run the next block and wait for a few minutes for the calculation to finish.

You can get gmpy2 on Windows from the following link: https://pypi.org/project/gmpy2/#files. On Linux, you can install it via your package manager (python3-gmpy2 on Apt, for example).

In [ ]:
from math import gcd
n = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
e = 0x10001
try:
    import gmpy2
    from gmpy2 import mpz
    sig_int = mpz(int.from_bytes(sig, "big"))
    m_int = mpz(int.from_bytes(unhexlify(padded_m), "big"))
    p_test = gmpy2.gcd(???)
except (ImportError, ModuleNotFoundError) as error:
    print("gmpy2 not found, falling back to Python")
    sig_int = int.from_bytes(sig, "big")
    padded_m_int = int.from_bytes(unhexlify(padded_m), "big")
    p_test = gcd(???)
    
print(hex(p_test))

We should now have either $p$ or $q$! We can get the other prime by simply dividing $n$ by the one we have.

In [ ]:
q_test = n//p_test
print(hex(q_test))

The $d$ calculation is a bit more complicated. Here's some code to derive it from $q$, $p$, and $e$:

In [ ]:
phi = (q_test - 1)*(p_test - 1)
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
        gcd = b
    return gcd, x, y

gcd, d, b = egcd(e, phi)

print(hex(d))

Let's sign the original message and see if we can verify it with our original verifier:

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 

from Crypto.Hash import SHA256

private_key = RSA.construct((n, e, int(d), int(p_test), int(q_test)))
private_signer = PKCS1_v1_5.new(private_key) 
new_sig = private_signer.sign(hash_object)
print(sig)

new_sig_check = signer.verify(hash_object, new_sig)
print(new_sig_check)

assert new_sig_check, "Failed to verify signature on device. Got: {}".format(newout)

Now that you've seen the attack work, you might want to try doing the falut in the other half of the RSA calculation to see if you can get the other prime number back.

## Caveats to the Attack

The crypto implementation we're attack isn't actually vulnerable to this attack without some additional glitching. This is because it verifies the signature is valid before sending it off. With a more complicated glitch setup, we could try glitching past it, but this is outside the scope of this lab. Some ideas include modifying the ChipWhisperer FPGA code to generate a second glitch, using a second ChipWhisperer (easiest if you were voltage glitching), and using a second trigger and trying to rearm between the two signature encryptions. You might even be able to increase the repeat and glitch near the end of the second encryption algorithm to glitch past both.

Instead though, we copied some of the functions and commented out the following signature check:

```C
    /* Compare in constant time just in case */
    for( diff = 0, i = 0; i < ctx->len; i++ )
        diff |= verif[i] ^ sig[i];
    diff_no_optimize = diff;

    if( diff_no_optimize != 0 )
    {
        ret = MBEDTLS_ERR_RSA_PRIVATE_FAILED;
        goto cleanup;
    }
```

## Conclusions & Next Steps

You've seen in previous labs that there are very powerful fault attacks on symmetric cryptographic algorithms that can be used to recover an AES key with a few faults. From this one, you've seen an even more powerful attack is possible on the asymmetric cryptographic algorithm RSA-CRT that can recover the plaintext in a single fault. 